### Packages

In [8]:
%pip install arch
%pip install scikit-learn
%pip install yfinance

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: arch in c:\users\heber\appdata\local\programs\python\python311\lib\site-packages (6.2.0)




[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
import numpy as np
import pandas as pd
import scipy #módulos para optimización, álgebra lineal, estadísticas, procesamiento de señales, interpolación, transformadas de Fourier, entre otros.
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn #herramientas y algoritmos simples y eficientes para la minería y el análisis de datos
from statsmodels.tsa.arima_model import ARIMA
from arch import arch_model
import seaborn as sns
import yfinance
import warnings
warnings.filterwarnings("ignore")
sns.set()

### Loading the data

In [10]:
raw_data = yfinance.download (tickers = "^GSPC ^FTSE ^N225 ^GDAXI", start = "1994-01-07", end = "2023-09-23", 
                              interval = "1d", group_by = 'ticker', auto_adjust = True, threads = True)

[*********************100%%**********************]  4 of 4 completed


In [12]:
df_comp = raw_data.copy()

In [13]:
df_comp['spx'] = df_comp['^GSPC'].Close[:]
df_comp['dax'] = df_comp['^GDAXI'].Close[:]
df_comp['ftse'] = df_comp['^FTSE'].Close[:]
df_comp['nikkei'] = df_comp['^N225'].Close[:]

In [14]:
df_comp = df_comp.iloc[1:]
del df_comp['^N225']
del df_comp['^GSPC']
del df_comp['^GDAXI']
del df_comp['^FTSE']
df_comp=df_comp.asfreq('b')
df_comp=df_comp.fillna(method='ffill')

### Creating Returns

In [15]:
df_comp['ret_spx'] = df_comp.spx.pct_change(1)*100
df_comp['ret_ftse'] = df_comp.ftse.pct_change(1)*100
df_comp['ret_dax'] = df_comp.dax.pct_change(1)*100
df_comp['ret_nikkei'] = df_comp.nikkei.pct_change(1)*100

### Splitting the Data

In [16]:
size = int(len(df_comp)*0.8)
df, df_test = df_comp.iloc[:size], df_comp.iloc[size:]

### Fitting a Model

In [18]:
%pip install pmdarima
#Búsqueda automática de los parámetros óptimos del modelo ARIMA
from pmdarima.arima import auto_arima


[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     -------------------------------------- 566.2/566.2 kB 1.1 MB/s eta 0:00:00
     ---------------------------------------- 2.8/2.8 MB 1.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [19]:
model_auto = auto_arima(df.ret_ftse[1:])

In [20]:
#ARIMA(4,0,5): Esto indica que el modelo tiene componentes autoregresivas (AR) de orden 4 y medias móviles (MA) de orden 5
#(0,0,0): Los términos (0,0,0) indican que no hay componentes estacionales en el modelo. 
#[0] intercept: Hay un término de intercepto en el modelo. Valor esperado de la serie temporal cuando todas las otras variables son cero.
model_auto

ARIMA(order=(4, 0, 5), scoring_args={}, suppress_warnings=True)

In [21]:
model_auto.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                 6199
Model:               SARIMAX(4, 0, 5)   Log Likelihood               -9468.378
Date:                Tue, 10 Oct 2023   AIC                          18958.757
Time:                        00:52:50   BIC                          19032.810
Sample:                    01-11-1994   HQIC                         18984.431
                         - 10-13-2017                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.0266      0.019      1.427      0.153      -0.010       0.063
ar.L1          0.0382      0.076      0.501      0.616      -0.111       0.187
ar.L2         -0.5738      0.077     -7.442      0.000      -0.725      -0.423
ar.L3         -0.1614      0.070     -2.314      0.021      -0.298      -0.025
ar.L4          0.3043      0.074      4.113      0.000       0.159       0.449
ma.L1         -0.0576      0.076     -0.757      0.449      -0.207       0.092
ma.L2          0.5245      0.078      6.714      0.000       0.371       0.678
ma.L3          0.0799      0.068      1.173      0.241      -0.054       0.213
ma.L4         -0.3065      0.073     -4.188      0.000      -0.450      -0.163
ma.L5         -0.0889      0.009    -10.398      0.000      -0.106      -0.072
sigma2         1.2444      0.012    102.562      0.000       1.221       1.268
===================================================================================
Ljung-Box (L1) (Q):                   0.00   Jarque-Bera (JB):              8324.63
Prob(Q):                              0.96   Prob(JB):                         0.00
Heteroskedasticity (H):               0.89   Skew:                            -0.20
Prob(H) (two-sided):                  0.01   Kurtosis:                         8.66
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

### Important Arguments

In [22]:
model_auto = auto_arima(df_comp.ret_ftse[1:], exogenous = df_comp[['ret_spx', 'ret_dax', 'ret_nikkei']][1:], m = 5,
                       max_order = None, max_p = 7, max_q = 7, max_d = 2, max_P = 4, max_Q = 4, max_D = 2,
                       maxiter = 50, alpha = 0.05, n_jobs = -1, trend = 'ct', information_criterion = 'oob',
                       out_of_sample = int(len(df_comp)*0.2))


# !!! Important Note: In pdmarima v1.5.2, out_of_sample_size is replaced with out_of_sample, so make sure to use the latter!


# exogenous -> factores externos (por ejemplo, otras series de tiempo)
# m -> duración del ciclo estacional
# max_order -> cantidad máxima de variables a utilizar en la regresión (p + q)
# max_p -> componentes maximos de AR
# max_q -> componentes maximos de MA
# max_d -> integraciones maximas
# maxiter -> iteraciones máximas que le estamos dando al modelo para hacer converger los coeficientes (se vuelve más difícil a medida que aumenta el orden)
# alpha -> nivel de significancia, el valor predeterminado es 5%, que deberíamos usar la mayor parte del tiempo
# n_jobs -> cuántos modelos caben a la vez (-1 indica "tantos como sea posible")
# trend -> "ct" usually
# information_criterion -> 'aic', 'aicc', 'bic', 'hqic', 'oob' 
#        (Akaike Information Criterion, Corrected Akaike Information Criterion,
#        Bayesian Information Criterion, Hannan-Quinn Information Criterion, or
#        "out of bag"--for validation scoring--respectively)
# out_of_smaple -> valida la selección del modelo (pase todo el conjunto de datos y establezca el 20% como out_of_sample_size)

In [23]:
model_auto.summary()
#(0, 0, [1, 2], 5) indica que hay un componente estacional con un período de 5 observaciones. 
#El [1, 2] Medias móviles estacionales (SMA), el modelo incluye términos de medias móviles estacionales de órdenes 1 y 2. 
#El 5 representa el período estacional.

<class 'statsmodels.iolib.summary.Summary'>
"""
                                       SARIMAX Results                                        
==============================================================================================
Dep. Variable:                                      y   No. Observations:                 7749
Model:             SARIMAX(3, 0, 3)x(0, 0, [1, 2], 5)   Log Likelihood              -11754.388
Date:                                Tue, 10 Oct 2023   AIC                          23530.776
Time:                                        01:01:23   BIC                          23607.284
Sample:                                    01-11-1994   HQIC                         23557.005
                                         - 09-22-2023                                         
Covariance Type:                                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.0258      0.027      0.943      0.345      -0.028       0.079
drift      -1.855e-06   6.25e-06     -0.297      0.766   -1.41e-05    1.04e-05
ar.L1         -0.3025      0.071     -4.242      0.000      -0.442      -0.163
ar.L2         -0.2583      0.071     -3.640      0.000      -0.397      -0.119
ar.L3          0.3823      0.073      5.272      0.000       0.240       0.524
ma.L1          0.2825      0.068      4.135      0.000       0.149       0.416
ma.L2          0.2148      0.070      3.089      0.002       0.078       0.351
ma.L3         -0.4611      0.070     -6.629      0.000      -0.597      -0.325
ma.S.L5       -0.0353      0.007     -4.791      0.000      -0.050      -0.021
ma.S.L10      -0.0114      0.007     -1.536      0.124      -0.026       0.003
sigma2         1.2194      0.009    129.169      0.000       1.201       1.238
===================================================================================
Ljung-Box (L1) (Q):                   0.00   Jarque-Bera (JB):             18528.00
Prob(Q):                              0.99   Prob(JB):                         0.00
Heteroskedasticity (H):               0.76   Skew:                            -0.32
Prob(H) (two-sided):                  0.00   Kurtosis:                        10.55
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""